
#ResNet101 on Cifar-10

##Import libraries


In [93]:
from copy import deepcopy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD,Adam,lr_scheduler
from torch.utils.data import random_split
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset
import torchvision
from torchvision import transforms, models, datasets

##Cifar-10 Dataset

In [94]:
train_indices = torch.arange(0, 49000)
valid_indices = torch.arange(49000, 50000)


train_and_valid = datasets.CIFAR10(root='data', 
                                   train=True, 
                                   transform=transforms.ToTensor(),
                                   download=True)

train_dataset = Subset(train_and_valid, train_indices)
valid_dataset = Subset(train_and_valid, valid_indices)


test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=transforms.ToTensor())

Files already downloaded and verified


##Cifar10 Dataloaders

In [95]:
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=64,
                          num_workers=8,
                          shuffle=True)

valid_loader = DataLoader(dataset=valid_dataset, 
                          batch_size=64,
                          num_workers=8,
                          shuffle=False)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=64,
                         num_workers=8,
                         shuffle=False)

## Model ResNet101

In [96]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        base = models.resnet101(pretrained=True)
        self.base = nn.Sequential(*list(base.children())[:-1])
        in_features = base.fc.in_features
        self.drop = nn.Dropout()
        self.final = nn.Linear(in_features,10)
    
    def forward(self,x):
        x = self.base(x)
        x = self.drop(x.view(-1,self.final.in_features))
        return self.final(x)
    
model = Model().cuda()

In [97]:
criterion = nn.CrossEntropyLoss()
param_groups = [
    {'params':model.base.parameters(),'lr':.0001},
    {'params':model.final.parameters(),'lr':.001}
]
optimizer = Adam(param_groups)
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)
states = {}

## Training

In [98]:
best_val_acc = -1000
best_val_model = None
for epoch in range(5):  
    model.train(True)
    running_loss = 0.0
    running_acc = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.cuda(),labels.cuda()

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item() * inputs.size(0)
        out = torch.argmax(outputs.detach(),dim=1)
        assert out.shape==labels.shape
        running_acc += (labels==out).sum().item()
    print(f"Train loss {epoch+1}: {running_loss/len(train_dataset)},Train Acc:{running_acc*100/len(train_dataset)}%")
    
    correct = 0
    model.train(False)
    with torch.no_grad():
        for inputs,labels in valid_loader:
            out = model(inputs.cuda()).cpu()
            out = torch.argmax(out,dim=1)
            acc = (out==labels).sum().item()
            correct += acc
    print(f"Val accuracy:{correct*100/len(valid_dataset)}%")
    if correct>best_val_acc:
        best_val_acc = correct
        best_val_model = deepcopy(model.state_dict())
    lr_scheduler.step()
    
print('Finished Training')  

Train loss 1: 0.9817252762560942,Train Acc:66.42244897959183%
Val accuracy:76.8%
Train loss 2: 0.44645514609862347,Train Acc:85.01020408163265%
Val accuracy:82.3%
Train loss 3: 0.3447384761279943,Train Acc:88.27755102040817%
Val accuracy:81.9%
Train loss 4: 0.32756214544724443,Train Acc:88.91836734693878%
Val accuracy:82.4%
Train loss 5: 0.32074569557151017,Train Acc:89.1%
Val accuracy:82.1%
Finished Training


## Testing

In [99]:
correct = 0
model.load_state_dict(best_val_model)
model.train(False)
with torch.no_grad():
    for inputs,labels in test_loader:
        out = model(inputs.cuda()).cpu()
        out = torch.argmax(out,dim=1)
        acc = (out==labels).sum().item()
        
        correct += acc
print(f"Test accuracy: {correct*100/len(test_dataset)}%")

Test accuracy: 82.45%
